In [1]:
import sys
import pyzed.sl as sl
import cv2

import math
import numpy as np
import random
from matplotlib import pyplot as plt
import time

import get_clear_hand_mask
import find_fingertip
import tracking
import touched_detection
import socket_sender


In [2]:
def candy_edge_with_skin_detect(image):
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb) # 把圖像轉換到YUV色域
    (y, cr, cb) = cv2.split(ycrcb) # 圖像分割, 分別獲取y, cr, br通道圖像
    # 高斯濾波, cr 是待濾波的源圖像數據, (5,5)是值窗口大小, 0 是指根據窗口大小來計算高斯函數標準差
    kernel_size = 5
    cr1 = cv2.GaussianBlur(cr, (kernel_size, kernel_size), 0) # 對cr通道分量進行高斯濾波
    # 根據OTSU算法求圖像閾值, 對圖像進行二值化_, 
    _ ,skin1 = cv2.threshold(cr1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) 
    
    cv2.imshow("Skin Cr+OSTU", skin1 )
    return skin1

# Transform Data

In [3]:
def send_fingertips_pos(fingertips, threed_points):
    data = ""
    temp = ['1 ']
    for i,tip in enumerate(fingertips):
        (u, v) = (int(tip[0]), int(tip[1]))
        temp.append("%d %d "%(u, v)) # speedup the string append
    data = data.join(temp)
    print('send fingertips pos:', data)
    socket_sender.send(data)
    
def send_touch_events(touch_flag):
    data = ""
    temp = ['2 ']
    for i,touch in enumerate(touch_flag):
        temp.append("%d "%(touch)) # speedup the string append
    data = data.join(temp)
    print('send touch flag:', data)
    socket_sender.send(data)

In [4]:
def main():

    init = sl.InitParameters()
    init.camera_resolution = sl.RESOLUTION.RESOLUTION_HD720
    init.depth_mode = sl.DEPTH_MODE.DEPTH_MODE_PERFORMANCE
    init.depth_minimum_distance = 150 # Set the minimum depth perception distance to 15cm
    init.coordinate_units = sl.UNIT.UNIT_MILLIMETER#sl.UNIT.UNIT_METER  #sl.UNIT.UNIT_MILLIMETER
#     print(init.depth_mode)#PERFORMANCE

    Streaming = False
    if(Streaming) : 
        #read from streaming
        sys.argv[1] = '127.0.0.1'
        if (len(sys.argv) > 1) :
            ip = sys.argv[1]
            init.set_from_stream(ip,30000)
        else :
            print('Usage : python3 streaming_receiver.py ip')
            exit(1)
    else:   
        #read from SVO file
        filepath = 'record_1028_6.svo'#record_1028_6.svo
        print("Reading SVO file: {0}".format(filepath))
        init = sl.InitParameters(svo_input_filename=filepath,svo_real_time_mode=False)
    
    

    cam = sl.Camera()
    status = cam.open(init)
    if status != sl.ERROR_CODE.SUCCESS:
        print(repr(status))
        exit(1)

    runtime = sl.RuntimeParameters()
    runtime.sensing_mode = sl.SENSING_MODE.SENSING_MODE_FILL 
    
    image = sl.Mat() #image
    depth = sl.Mat() #depth map
    depth_for_display = sl.Mat() #depth map,scale its values to [0, 255]
    point_cloud = sl.Mat() #colored point cloud.
    confidence_map = sl.Mat() # confidence_map  [0 ,100], 0 :the best confidence value; 100 :the most unconfident one.

    key = ''
    First = True
    print("  Quit : CTRL+C\n")
    while key != 113:
        err = cam.grab(runtime)
        if (err == sl.ERROR_CODE.SUCCESS) :
            # Retrieve left image
            cam.retrieve_image(image, sl.VIEW.VIEW_LEFT)
            cam.retrieve_image(depth_for_display, sl.VIEW.VIEW_DEPTH)
            
            # Load Image data into a numpy array
            RGB_image = image.get_data()
            gray_image = cv2.cvtColor(RGB_image, cv2.COLOR_BGR2GRAY)
            
            
            # Retrieve depth map. Depth is aligned on the left image
            cam.retrieve_measure(depth, sl.MEASURE.MEASURE_DEPTH)
            # Load depth data into a numpy array
            depth_data = depth.get_data()
            
            # Retrieve colored point cloud. Point cloud is aligned on the left image.
            cam.retrieve_measure(point_cloud, sl.MEASURE.MEASURE_XYZRGBA)
            # Load point cloud into a numpy array
            points3D_color = point_cloud.get_data()
            points3D = points3D_color[:,:,:3] # only need x,y,z 
#             points3D = np.where(points3D == np.nan, 1111, points3D)# replace nan to 1111111
            
            
            #get confidence index : index which value is not NAN 
            Confidence_index = np.argwhere(~np.isnan(depth_data[200:520,300:980]))#center region of image
#             Confidence_index = np.argwhere(~np.isnan(depth_data[:520,300:980]))
            
#             # Retrieve confidence_map.
#             cam.retrieve_measure(confidence_map, sl.MEASURE.MEASURE_CONFIDENCE)
#             # Load confidence_map into a numpy array
#             confidence_img = confidence_map.get_data()
#             confidence_index = np.argwhere(confidence_img <= 60)

            print("Start computing...")
            #####
            if First == True:
                #RANSAM to get surface plane
                surface_plane, depthImg, plane_mask = get_clear_hand_mask.RANSAM(points3D, Confidence_index, ransac_iteration = 30, inliner_threshold = 10)
                First = False
                print("firstly RANSAM")
#                 socket_sender.send("0 firstly RANSAM")
                cv2.imshow("plane_mask", plane_mask.astype(np.uint8)*255)
                
            else:
            
                s_time = time.time()
                #Ransam every times ???
#                 Confidence_index = np.argwhere(plane_mask == True)
#                 surface_plane, depthImg, plane_mask = get_clear_hand_mask.RANSAM(points3D, Confidence_index, ransac_iteration = 10, inliner_threshold = 10)

                #
                depthImg = get_clear_hand_mask.get_depth_map(points3D, surface_plane)
                print('depth: %.4f'%(time.time() - s_time))
        
                #Canny edge map(infrared image) + threshold based edge map(depth image)
                t_time = time.time()
                Cannyedges, Threshold_based_edge, Edge_map, grayImage = get_clear_hand_mask.get_edge_map(gray_image, depthImg)
                print('edges: %.4f'%(time.time() - t_time))
                
#                 Test skin_detect than Canny edge
                t_time = time.time()
                skinImg = candy_edge_with_skin_detect(RGB_image)
                print('skinImg: %.4f'%(time.time() - t_time))

#                 #Get hight region by Hight and record its position
                t_time = time.time()
                High_region_Image, high_region_list = get_clear_hand_mask.get_high_region(depthImg, plane_mask)
                print('High_region: %.4f'%(time.time() - t_time))
        
#                 #Get Hand mask by Flood fill from high region position with Edge map and check wheather Flood_fill_with_edge_reasonable
                t_time = time.time()
                Hand_mask_Image = get_clear_hand_mask.get_reasonable_Hand_mask(Edge_map, high_region_list, High_region_Image, points3D)
                print('Hand_mask: %.4f'%(time.time() - t_time))
        
#                 #find fingertip
                t_time = time.time()
                ##skinImg
                #Hand_mask_Image = skinImg
                cnt, contours_image, hand_center, fingertips = find_fingertip.find_fingertip(Hand_mask_Image)#Hand_mask_Image skinImg
                print('fingertips: %.4f'%(time.time() - t_time))
        
#                 # tracking user hand and fingertips
                t_time = time.time()
                hands_hand, fingertips_pos, tracking_image = tracking.hand_tracking(hand_center, fingertips, points3D, contours_image)
                print('tracking: %.4f'%(time.time() - t_time))
            
#                 # detecte fingers whenther touching the surface
                t_time = time.time()
                touched_flag, touching_detection_image = touched_detection.touching_detection(fingertips_pos, Hand_mask_Image, depthImg, contours_image)
                send_touch_events(touched_flag)
                print('touched_detection: %.4f'%(time.time() - t_time))
                
                print('computing time:', (time.time()-s_time))
        
            #####
                #Check result
                cv2.imshow("Depth", depthImg.astype(np.uint8))
# #                 cv2.imshow("plane_mask", plane_mask.astype(np.uint8)*255)
# #                 cv2.imshow("Cannyedges", Cannyedges)
#                 cv2.imshow("Threshold_based_edge", Threshold_based_edge.astype(np.uint8)*255)
                cv2.imshow("Edge_map", Edge_map.astype(np.uint8)*255)
# #                 cv2.imshow("High_region_Image", High_region_Image.astype(np.uint8)*255)
# #                 cv2.imshow("Hand_mask_Image", Hand_mask_Image.astype(np.uint8)*255)
                cv2.imshow("tracking_image", tracking_image)
                cv2.imshow("touching_detection_image", touching_detection_image)
                
#                 cv2.imshow("ZED_gray", grayImage)
    
            cv2.imshow("ZED", RGB_image)
#             cv2.imshow("Depth img", depth_for_display.get_data())

            key = cv2.waitKey(1)
        else :
            key = cv2.waitKey(1)
        
        if key == 113:
            cv2.destroyAllWindows()
            break
        
    cam.close()
    

In [5]:
if __name__ == "__main__":
    main()

Reading SVO file: record_1028_6.svo
  Quit : CTRL+C

Start computing...
Inliner Number
 654539
Inliner plane
 [ 2.30160393e+02 -2.03063899e+03 -4.11124400e+04  1.91054844e+07]
firstly RANSAM
Start computing...
depth: 0.0349
*get threshold edge: 0.0499 s
edges: 0.0568
skinImg: 0.0090
pos :  (959, 505)
High_region: 0.0189
Dist:  719.3949107430974
Hand_mask: 0.0060
fingertips: 0.0120
tracking: 0.0020
31.562721989079257 27.729918152241 3.8328038368382558
33.67752688308172 27.203093534525994 6.474433348555724
33.26298286643757 27.555056680444903 5.707926185992665
33.18614013778578 24.671600755697224 8.514539382088554
touched pos: (793, 210)
touched pos: (753, 251)
touched pos: (754, 495)
touched pos: (896, 255)
send touch flag: 2 1 1 0 1 0 1 0 0 0 0 
create socket successful
touched_detection: 0.0030
computing time: 0.1436164379119873
Start computing...
depth: 0.0369
*get threshold edge: 0.0519 s
edges: 0.0608
skinImg: 0.0060
pos :  (958, 501)
High_region: 0.0170
Dist:  723.3560713943384
Ha

tracking: 0.0010
31.261827070599235 28.83327528230644 2.4285517882927934
34.15777570237756 26.16883714460285 7.988938557774713
30.93110034262473 29.19233195247754 1.7387683901471895
touched pos: (893, 252)
touched pos: (818, 399)
touched pos: (865, 571)
send touch flag: 2 1 0 1 1 0 0 0 0 0 0 
touched_detection: 0.0020
computing time: 0.1346421241760254
Start computing...
depth: 0.0349
*get threshold edge: 0.0509 s
edges: 0.0568
skinImg: 0.0060
pos :  (1026, 518)
High_region: 0.0189
Dist:  799.3220791754643
Hand_mask: 0.0070
fingertips: 0.0100
tracking: 0.0020
31.65795482360408 28.757748181161183 2.900206642442896
32.078155072955475 28.623325467724296 3.454829605231179
31.998013078743234 27.443330482189342 4.554682596553892
31.12618140906018 28.453031100561827 2.6731503084983537
touched pos: (904, 255)
touched pos: (987, 244)
touched pos: (823, 331)
touched pos: (867, 564)
send touch flag: 2 1 1 1 1 0 0 0 0 0 0 
touched_detection: 0.0020
computing time: 0.13962721824645996
Start computi

tracking: 0.0010
34.03928702926247 24.29193769151896 9.747349337743508
31.203587002918113 28.28539931560519 2.9181876873129227
30.727291512262145 28.94148959166002 1.7858019206021254
31.8935677733519 27.908718444445665 3.984849328906236
31.62926527819735 28.704190686893423 2.925074591303929
touched pos: (940, 330)
touched pos: (920, 422)
touched pos: (940, 480)
touched pos: (1099, 313)
touched pos: (1044, 281)
send touch flag: 2 1 1 1 1 1 0 0 0 0 0 
touched_detection: 0.0030
computing time: 0.13663482666015625
Start computing...
depth: 0.0339
*get threshold edge: 0.0489 s
edges: 0.0568
skinImg: 0.0050
pos :  (1056, 540)
High_region: 0.0180
Dist:  821.0452844016596
Hand_mask: 0.0060
fingertips: 0.0100
tracking: 0.0010
31.864328143149393 28.403596832478115 3.460731310671278
30.768409432090575 29.278963067780687 1.489446364309888
32.060325521333645 28.38967969114065 3.6706458301929956
touched pos: (999, 282)
touched pos: (942, 334)
touched pos: (947, 482)
touched pos: (1105, 317)
send tou

pos :  (228, 392)
High_region: 0.0209
Dist:  371.57326934918325
Hand_mask: 0.0050
fingertips: 0.0090
tracking: 0.0020
35.05829093732535 29.020417847620575 6.037873089704778
30.08418511190552 29.91068455416057 0.17350055774494777
30.110683354261646 29.914779511530227 0.19590384273141837
touched pos: (414, 644)
touched pos: (312, 37)
touched pos: (386, 154)
send touch flag: 2 1 1 1 0 0 0 0 0 0 0 
touched_detection: 0.0020
computing time: 0.1376323699951172
Start computing...
depth: 0.0349
*get threshold edge: 0.0489 s
edges: 0.0568
skinImg: 0.0050
pos :  (215, 403)
High_region: 0.0199
Dist:  366.23506050629015
Hand_mask: 0.0050
fingertips: 0.0070
tracking: 0.0010
30.088303248867334 29.932626361067662 0.1556768877996717
touched pos: (345, 165)
send touch flag: 2 1 0 0 0 0 0 0 0 0 0 
touched_detection: 0.0020
computing time: 0.1326456069946289
Start computing...
depth: 0.0369
*get threshold edge: 0.0489 s
edges: 0.0559
skinImg: 0.0060
pos :  (192, 436)
High_region: 0.0199
Dist:  372.342135

High_region: 0.0180
Dist:  773.7460319763959
Hand_mask: 0.0070
fingertips: 0.0100
tracking: 0.0010
31.836573203820738 22.137038246373994 9.699534957446744
32.48626681275862 28.31557879888467 4.170688013873953
touched pos: (904, 235)
touched pos: (862, 357)
send touch flag: 2 1 1 0 0 0 0 0 0 0 0 
touched_detection: 0.0020
computing time: 0.13164758682250977
Start computing...
depth: 0.0349
*get threshold edge: 0.0479 s
edges: 0.0559
skinImg: 0.0070
pos :  (1027, 528)
High_region: 0.0170
Dist:  720.5713202762507
Hand_mask: 0.0060
fingertips: 0.0090
tracking: 0.0020
31.27123595690845 25.991066670728667 5.280169286179781
32.57712904405531 27.08253966130663 5.494589382748682
149.73801811585898 147.65964117900558 2.0783769368534024
32.90048675457983 23.833694376712998 9.066792377866832
touched pos: (893, 239)
touched pos: (853, 371)
touched pos: (1279, 696)
touched pos: (998, 281)
send touch flag: 2 1 1 0 1 1 0 0 0 0 0 
touched_detection: 0.0030
computing time: 0.13464021682739258
Start comp

...

pos :  (613, 340)
High_region: 0.0229
Dist:  556.5368700746917
Hand_mask: 0.0070
fingertips: 0.0090
tracking: 0.0020
34.1106533104209 26.185187718248194 7.925465592172706
touched pos: (529, 660)
send touch flag: 2 1 0 0 0 0 0 0 0 0 0 
touched_detection: 0.0020
computing time: 0.14162135124206543
Start computing...
depth: 0.0369
*get threshold edge: 0.0499 s
edges: 0.0578
skinImg: 0.0060
pos :  (609, 338)
High_region: 0.0209
Dist:  548.845210503928
Hand_mask: 0.0080
fingertips: 0.0110
tracking: 0.0010
34.71046332969906 26.404305531236524 8.306157798462536
touched pos: (533, 661)
send touch flag: 2 1 0 0 0 0 0 0 0 0 0 
touched_detection: 0.0020
computing time: 0.14461302757263184
Start computing...
depth: 0.0359
*get threshold edge: 0.0529 s
edges: 0.0588
skinImg: 0.0050
pos :  (609, 339)
High_region: 0.0219
Dist:  551.0678573084713
Hand_mask: 0.0070
fingertips: 0.0120
tracking: 0.0010
51.940489023332056 52.261319699100056 -0.3208306757680006
touched pos: (3, 605)
send touch flag: 2 1 0 

High_region: 0.0239
Dist:  533.9996777818227
Hand_mask: 0.0080
fingertips: 0.0100
tracking: 0.0010
32.34568837464206 27.887506533851933 4.458181840790125
30.44778697490691 29.4516641509927 0.996122823914213
30.94622619116057 29.25038283337786 1.6958433577827101
touched pos: (729, 663)
touched pos: (1219, 404)
touched pos: (791, 640)
send touch flag: 2 1 1 1 0 0 0 0 0 0 0 
touched_detection: 0.0020
computing time: 0.14261913299560547
Start computing...
depth: 0.0359
*get threshold edge: 0.0479 s
edges: 0.0559
skinImg: 0.0050
pos :  (577, 325)
High_region: 0.0229
Dist:  531.6909121619973
Hand_mask: 0.0080
fingertips: 0.0100
tracking: 0.0010
31.422393122555537 28.208785117919614 3.2136080046359226
30.627788903934242 29.42858104533183 1.1992078586024135
30.986400598678852 29.247787632756353 1.7386129659224991
touched pos: (742, 662)
touched pos: (1218, 406)
touched pos: (791, 641)
send touch flag: 2 1 1 1 0 0 0 0 0 0 0 
touched_detection: 0.0020
computing time: 0.1406242847442627
Start com

Hand_mask: 0.0070
fingertips: 0.0100
tracking: 0.0010
31.6938286473908 27.598288184397806 4.095540462992993
30.747681611922296 29.018364993068612 1.7293166188536837
31.070480444817434 28.916071845633525 2.1544085991839097
30.83510531880305 29.735605138920995 1.0995001798820532
touched pos: (751, 661)
touched pos: (1225, 413)
touched pos: (788, 645)
touched pos: (1279, 301)
send touch flag: 2 1 1 1 1 0 0 0 0 0 0 
touched_detection: 0.0030
computing time: 0.14760518074035645
Start computing...
depth: 0.0369
*get threshold edge: 0.0509 s
edges: 0.0588
skinImg: 0.0060
pos :  (579, 326)
High_region: 0.0239
Dist:  528.8128287561634
Hand_mask: 0.0080
fingertips: 0.0120
tracking: 0.0010
31.393951277848558 28.733669242535644 2.660282035312914
30.91465565995412 25.759617354800575 5.155038305153546
31.23960402074481 29.074454088829768 2.1651499319150425
31.069818455786425 29.761927873893114 1.3078905818933109
touched pos: (756, 657)
touched pos: (1228, 408)
touched pos: (787, 642)
touched pos: (1

depth: 0.0389
*get threshold edge: 0.0479 s
edges: 0.0539
skinImg: 0.0050
pos :  (606, 336)
High_region: 0.0229
Dist:  466.83442265838374
Hand_mask: 0.0060
fingertips: 0.0100
tracking: 0.0020
30.861879086260117 29.599652019341516 1.2622270669186015
touched pos: (1279, 95)
send touch flag: 2 0 1 0 0 0 0 0 0 0 0 
touched_detection: 0.0010
computing time: 0.13962674140930176
Start computing...
depth: 0.0349
*get threshold edge: 0.0479 s
edges: 0.0549
skinImg: 0.0060
pos :  (612, 340)
High_region: 0.0219
Dist:  751.5938169882374
Hand_mask: 0.0070
fingertips: 0.0110
tracking: 0.0020
30.67035196929683 29.369043743240802 1.3013082260560296
touched pos: (1249, 15)
send touch flag: 2 1 0 0 0 0 0 0 0 0 0 
touched_detection: 0.0010
computing time: 0.13962697982788086
Start computing...
depth: 0.0349
*get threshold edge: 0.0499 s
edges: 0.0559
skinImg: 0.0080
pos :  (613, 340)
High_region: 0.0229
Dist:  472.73817451599916
Hand_mask: 0.0060
fingertips: 0.0100
tracking: 0.0010
32.06164005298186 26.9

Start computing...
depth: 0.0349
*get threshold edge: 0.0469 s
edges: 0.0519
skinImg: 0.0050
pos :  (612, 338)
High_region: 0.0219
Dist:  568.2206273937114
Hand_mask: 0.0070
fingertips: 0.0070
tracking: 0.0010
72.24656253717535 72.7544151968559 -0.5078526596805517
touched pos: (2, 575)
send touch flag: 2 1 0 0 0 0 0 0 0 0 0 
touched_detection: 0.0020
computing time: 0.13164806365966797
Start computing...
depth: 0.0359
*get threshold edge: 0.0469 s
edges: 0.0559
skinImg: 0.0050
pos :  (612, 338)
High_region: 0.0219
Dist:  575.0148630404406
Hand_mask: 0.0060
fingertips: 0.0090
tracking: 0.0010
send touch flag: 2 0 0 0 0 0 0 0 0 0 0 
touched_detection: 0.0010
computing time: 0.13663506507873535
Start computing...
depth: 0.0349
*get threshold edge: 0.0499 s
edges: 0.0559
skinImg: 0.0050
pos :  (608, 336)
High_region: 0.0259
Dist:  582.1183427827416
Hand_mask: 0.0070
fingertips: 0.0080
tracking: 0.0010
send touch flag: 2 0 0 0 0 0 0 0 0 0 0 
touched_detection: 0.0010
computing time: 0.13962

In [6]:
a = np.zeros((720,1280))
a[200:520,300:980] = 255
x = 50
a[x:x+35,x:x+35] = 255
x = 600
a[x:x+7,x:x+7] = 255
cv2.imshow("A",a.astype(np.uint8))
cv2.waitKey()
cv2.destroyAllWindows()


In [7]:
# It need to restart than run ==> socket.sendall would not break

In [8]:
t_time = time.time()
np.abs(a)>2
print('touched_detection: %.4f'%(time.time() - t_time))


t_time = time.time()
np.logical_or(a>2,a<2)
print('touched_detection: %.4f'%(time.time() - t_time))



x1 = np.logical_or(a,a<2)
x2 = np.logical_or(a,a>2)
x3 = np.logical_or(a,a<2)
x4 = np.logical_or(a,a>2)
x5 = np.logical_or(a,a<2)



t_time = time.time()
check_1_8 = np.array((x1, x2, x3, x4, x5))
Threshold_based_edge = np.logical_or.reduce(check_1_8)
print('touched_detection: %.4f'%(time.time() - t_time))

x1+ x2+ x3

touched_detection: 0.0040
touched_detection: 0.0010
touched_detection: 0.0020


array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])